In this notebook, I estimated the stability of the ICRF axes in the frame of ICRF3.

In [1]:
from astropy.table import Table, join, Column
from astropy import units as u
import numpy as np
# My modules
from my_progs.catalog.vsh_deg1_cor import vsh_deg01_fitting
from my_progs.catalog.vsh_deg2_cor import vsh_deg02_fitting, residual_calc02
from my_progs.catalog.write_output import print_vsh1_corr, print_vsh2_corr
from my_progs.catalog.read_icrf import read_icrf1, read_icrf2, read_icrf3
from my_progs.catalog.pos_diff import radio_cat_diff_calc

In [2]:
# Read ICRF1 data
icrf1 = read_icrf1()

# Read ICRF2 data
icrf2 = read_icrf2()

# Read ICRF3 S/X catalog
icrf3sx = read_icrf3(wv="sx")

# Read ICRF3 K catalog
icrf3k = read_icrf3(wv="k")

# Read ICRF3 X/Ka catalog
icrf3xka = read_icrf3(wv="xka")

# ICRF3 defining source list
alllist = Table(icrf3sx)
alllist.keep_columns(["iers_name", "type"])
deflist = alllist[alllist["type"] == "D"]

In [3]:
icrf13sx = radio_cat_diff_calc(icrf1, icrf3sx, "iers_name", label=["1", "3sx"])
icrf23sx = radio_cat_diff_calc(icrf2, icrf3sx, "iers_name", label=["2", "3sx"])
icrf3k3sx = radio_cat_diff_calc(icrf3k, icrf3sx, "iers_name", label=["3k", "3sx"])
icrf3xka3sx = radio_cat_diff_calc(icrf3xka, icrf3sx, "iers_name", label=["3xka", "3sx"])

In [4]:
# Use only defining sources
icrf13sxdef = join(icrf13sx, deflist, keys="iers_name")
icrf23sxdef = join(icrf23sx, deflist, keys="iers_name")
icrf3k3sxdef = join(icrf3k3sx, deflist, keys="iers_name")
icrf3xka3sxdef = join(icrf3xka3sx, deflist, keys="iers_name")

In [5]:
# ICRF1 - ICRF3 SX
# Transform columns into np.array
dra = np.array(icrf13sxdef["dra"])
ddec = np.array(icrf13sxdef["ddec"])
dra_err = np.array(icrf13sxdef["dra_err"])
ddec_err = np.array(icrf13sxdef["ddec_err"])
ra_rad = np.array(icrf13sxdef["ra"].to(u.radian))
dec_rad = np.array(icrf13sxdef["dec"].to(u.radian))
dra_ddec_cov = np.array(icrf13sxdef["dra_ddec_cov"])

# Transformation parameters
w1_all, sig1_all, corrcoef1_all = vsh_deg01_fitting(
    dra, ddec, ra_rad, dec_rad, dra_err, ddec_err,
    cov=dra_ddec_cov, elim_flag="None", fit_type="rotation")

# mas -> uas
w1 = w1_all * 1.e3
sig1 = sig1_all * 1.e3


# Print results
print("Estimates (%6d sources)\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "               Rotation [uas]               \n"
      "               x             y             z\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "    %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f\n"
      "----------------------------------------------"
      "----------------------------------------------\n" %
      (dra.size,
       w1[0], sig1[0], w1[1], sig1[1], w1[2], sig1[2]))

print("Correlation coefficient between parameters in fit")
print(corrcoef1_all)

Estimates (   173 sources)
--------------------------------------------------------------------------------------------
               Rotation [uas]               
               x             y             z
--------------------------------------------------------------------------------------------
     +11 +/-  22    +2 +/-  22   +34 +/-  20
--------------------------------------------------------------------------------------------

Correlation coefficient between parameters in fit
[[1.         0.05403537 0.04380303]
 [0.05403537 1.         0.03850518]
 [0.04380303 0.03850518 1.        ]]


In [6]:
# ICRF2 - ICRF3 SX
# Transform columns into np.array
dra = np.array(icrf23sxdef["dra"])
ddec = np.array(icrf23sxdef["ddec"])
dra_err = np.array(icrf23sxdef["dra_err"])
ddec_err = np.array(icrf23sxdef["ddec_err"])
ra_rad = np.array(icrf23sxdef["ra"].to(u.radian))
dec_rad = np.array(icrf23sxdef["dec"].to(u.radian))
dra_ddec_cov = np.array(icrf23sxdef["dra_ddec_cov"])

# Transformation parameters
w1_all, sig1_all, corrcoef1_all = vsh_deg01_fitting(
    dra, ddec, ra_rad, dec_rad, dra_err, ddec_err,
    cov=dra_ddec_cov, elim_flag="None", fit_type="rotation")

# mas -> uas
w1 = w1_all * 1.e3
sig1 = sig1_all * 1.e3


# Print results
print("Estimates (%6d sources)\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "               Rotation [uas]               \n"
      "               x             y             z\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "    %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f\n"
      "----------------------------------------------"
      "----------------------------------------------\n" %
      (dra.size,
       w1[0], sig1[0], w1[1], sig1[1], w1[2], sig1[2]))

print("Correlation coefficient between parameters in fit")
print(corrcoef1_all)

Estimates (   296 sources)
--------------------------------------------------------------------------------------------
               Rotation [uas]               
               x             y             z
--------------------------------------------------------------------------------------------
     -10 +/-   7   +20 +/-   8    -1 +/-   7
--------------------------------------------------------------------------------------------

Correlation coefficient between parameters in fit
[[ 1.          0.00952247 -0.01532758]
 [ 0.00952247  1.          0.03275348]
 [-0.01532758  0.03275348  1.        ]]


In [7]:
# ICRF3 K - ICRF3 SX
# Transform columns into np.array
dra = np.array(icrf3k3sxdef["dra"])
ddec = np.array(icrf3k3sxdef["ddec"])
dra_err = np.array(icrf3k3sxdef["dra_err"])
ddec_err = np.array(icrf3k3sxdef["ddec_err"])
ra_rad = np.array(icrf3k3sxdef["ra"].to(u.radian))
dec_rad = np.array(icrf3k3sxdef["dec"].to(u.radian))
dra_ddec_cov = np.array(icrf3k3sxdef["dra_ddec_cov"])

# Transformation parameters
w1_all, sig1_all, corrcoef1_all = vsh_deg01_fitting(
    dra, ddec, ra_rad, dec_rad, dra_err, ddec_err,
    cov=dra_ddec_cov, elim_flag="None", fit_type="rotation")

# mas -> uas
w1 = w1_all * 1.e3
sig1 = sig1_all * 1.e3


# Print results
print("Estimates (%6d sources)\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "               Rotation [uas]               \n"
      "               x             y             z\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "    %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f\n"
      "----------------------------------------------"
      "----------------------------------------------\n" %
      (dra.size,
       w1[0], sig1[0], w1[1], sig1[1], w1[2], sig1[2]))

print("Correlation coefficient between parameters in fit")
print(corrcoef1_all)

Estimates (   193 sources)
--------------------------------------------------------------------------------------------
               Rotation [uas]               
               x             y             z
--------------------------------------------------------------------------------------------
     -10 +/-  10    -9 +/-  11    -5 +/-   7
--------------------------------------------------------------------------------------------

Correlation coefficient between parameters in fit
[[1.         0.0145123  0.00328077]
 [0.0145123  1.         0.04192732]
 [0.00328077 0.04192732 1.        ]]


In [8]:
# ICRF3 XKa - ICRF3 SX
# Transform columns into np.array
dra = np.array(icrf3xka3sxdef["dra"])
ddec = np.array(icrf3xka3sxdef["ddec"])
dra_err = np.array(icrf3xka3sxdef["dra_err"])
ddec_err = np.array(icrf3xka3sxdef["ddec_err"])
ra_rad = np.array(icrf3xka3sxdef["ra"].to(u.radian))
dec_rad = np.array(icrf3xka3sxdef["dec"].to(u.radian))
dra_ddec_cov = np.array(icrf3xka3sxdef["dra_ddec_cov"])

# Transformation parameters
w1_all, sig1_all, corrcoef1_all = vsh_deg01_fitting(
    dra, ddec, ra_rad, dec_rad, dra_err, ddec_err,
    cov=dra_ddec_cov, elim_flag="None", fit_type="rotation")

# mas -> uas
w1 = w1_all * 1.e3
sig1 = sig1_all * 1.e3


# Print results
print("Estimates (%6d sources)\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "               Rotation [uas]               \n"
      "               x             y             z\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "    %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f\n"
      "----------------------------------------------"
      "----------------------------------------------\n" %
      (dra.size,
       w1[0], sig1[0], w1[1], sig1[1], w1[2], sig1[2]))

print("Correlation coefficient between parameters in fit")
print(corrcoef1_all)

Estimates (   176 sources)
--------------------------------------------------------------------------------------------
               Rotation [uas]               
               x             y             z
--------------------------------------------------------------------------------------------
     -28 +/-  26   -25 +/-  27   -52 +/-  19
--------------------------------------------------------------------------------------------

Correlation coefficient between parameters in fit
[[1.         0.0389664  0.00169324]
 [0.0389664  1.         0.03830745]
 [0.00169324 0.03830745 1.        ]]
